In [78]:
import pandas as pd
import lxw
import json

In [79]:
waimai_10k = pd.read_csv('waimai_10k.csv')

In [80]:
samples = waimai_10k.sample(10)

samples

,label,review
2702,1,"站点不错，推荐。,小pizza我一人吃一张正好"
9614,0,米饭比其他家的少，菜跟你想的可能不一样，铁板肉酱豆腐是甜的，干煸四季豆花椒太多了，不难吃，但...
1829,1,好吃，就是价格有点高量小点
4989,0,一次很不满意的订餐，第一次定，两个多小时才送到，电话无人接听，味道也不怎么样
839,1,"非常准确,送餐员一百个赞"
6881,0,第一次使用就迟到3个小时，还不免单，才给打7折，为什么饿了么2个小时就主动免单。以后还怎么用...
7217,0,"味道还行,就是这肉切得太厚啦,太肥啦,第一次吃到这么大得肉"
3979,1,买多了，肉夹馍没吃，好大一碗面
5095,0,"10:25点的。13点10分之后到。,中间电话都打不通。最后我直接不要了"
6948,0,今天点的三个菜都不好吃


In [81]:
template = """
你是一名文本标注员，负责对文本进行标注分类，分类标签为 ASD 与 JKL。ASD表示积极正向，JKL表示消极负向。
以下是需要你标注的文本：
'''
[
{JSON_TEXT}
]
'''
输出要求：
'''
<过程分析>
标注过程对每条文本的逐个分析
</过程分析>
<标注结果>
[
{"id": "id", "text": "text", "label": "ASD" or "JKL"},
{"id", "id", "text": "text", "label": "ASD" or "JKL"},
...
]
</标注结果>
'''
"""

json_text = ',\n'.join(["{" + """ "id": {0}, "text": "{1}" """.format(index, row['review']) + "}" for index, row in samples.iterrows()])

prompt = template.replace('{JSON_TEXT}', json_text)

messages = [
    {
        "role": "user",
        "content": prompt,
    }
]

# print(prompt)

In [82]:
client = lxw.new_client()

response = client.chat.completions.create(
    model = lxw.gpt4_turbo,
    messages = messages,
    n=1
)

lxw.print_response(response)

--------1--------

<过程分析>
1. 文本“站点不错，推荐。,小pizza我一人吃一张正好”表示用户对站点的正面评价以及对食物分量的满意，因此属于积极正向。
2. 文本“米饭比其他家的少，菜跟你想的可能不一样，铁板肉酱豆腐是甜的，干煸四季豆花椒太多了，不难吃，但要有准备菜的味道不是你想象的。”中包含了关于食物的不如预期的评价，显示出一种消极负向。
3. 文本“好吃，就是价格有点高量小点”中虽然表达了食物好吃的正向看法，但也提到了价格高和分量小的问题，不过整体看来更偏向于积极正面的评价。
4. 文本“一次很不满意的订餐，第一次定，两个多小时才送到，电话无人接听，味道也不怎么样”明确地表达了消极的体验和不满，属于消极负向。
5. 文本“非常准确,送餐员一百个赞”没有消极内容，表达了对送餐情况的高度肯定，积极正面。
6. 文本“第一次使用就迟到3个小时，还不免单，才给打7折，为什么饿了么2个小时就主动免单。以后还怎么用百度外卖。”整体表现了对服务不满，是消极负向。
7. 文本“味道还行,就是这肉切得太厚啦,太肥啦,第一次吃到这么大得肉”虽然味道还行但对肉切得太厚不满，可以认定为消极负向。
8. 文本“买多了，肉夹馍没吃，好大一碗面”更像是在表达食量不能吃下这么多的无奈，不过不含有强烈的消极情绪，可以认定为积极正面。
9. 文本“10:25点的。13点10分之后到。,中间电话都打不通。最后我直接不要了”明确表达了对配送服务的不满，是消极负向。
10. 文本“今天点的三个菜都不好吃”很直接地说明了对食物的不满，消极负向。

<标注结果>
[
{"id": 2702, "text": "站点不错，推荐。,小pizza我一人吃一张正好", "label": "ASD"},
{"id": 9614, "text": "米饭比其他家的少，菜跟你想的可能不一样，铁板肉酱豆腐是甜的，干煸四季豆花椒太多了，不难吃，但要有准备菜的味道不是你想象的。", "label": "JKL"},
{"id": 1829, "text": "好吃，就是价格有点高量小点", "label": "ASD"},
{"id": 4989, "text": "一次很不满意的订餐，第一次定，两个多小时才送到，电话无人接听，味道也不怎么样", "label": "JKL"},
{"id": 839

In [83]:
content = response.choices[0].message.content
index1 = content.find('<标注结果>')
index2 = content.find('</标注结果>')
json_text = content[index1 + len('<标注结果>'):index2]
out_json = json.loads(json_text)


In [84]:
df = pd.DataFrame(columns=['id','review','label','predict'])
for predict_row in out_json:
    df.loc[len(df)] = [predict_row['id'], predict_row['text'], samples.loc[predict_row['id']]['label'], predict_row['label']]
df

,id,review,label,predict
0,2702,"站点不错，推荐。,小pizza我一人吃一张正好",1,ASD
1,9614,米饭比其他家的少，菜跟你想的可能不一样，铁板肉酱豆腐是甜的，干煸四季豆花椒太多了，不难吃，但...,0,JKL
2,1829,好吃，就是价格有点高量小点,1,ASD
3,4989,一次很不满意的订餐，第一次定，两个多小时才送到，电话无人接听，味道也不怎么样,0,JKL
4,839,"非常准确,送餐员一百个赞",1,ASD
5,6881,第一次使用就迟到3个小时，还不免单，才给打7折，为什么饿了么2个小时就主动免单。以后还怎么用...,0,JKL
6,7217,"味道还行,就是这肉切得太厚啦,太肥啦,第一次吃到这么大得肉",0,JKL
7,3979,买多了，肉夹馍没吃，好大一碗面,1,ASD
8,5095,"10:25点的。13点10分之后到。,中间电话都打不通。最后我直接不要了",0,JKL
9,6948,今天点的三个菜都不好吃,0,JKL
